In [ ]:
import pandas as pd
import numpy as np
import umap
import re
from collections import defaultdict
import plotly.express as px
import hdbscan
from sklearn.manifold import MDS
from sklearn.cluster import SpectralClustering
import math
import statistics as stats

In [ ]:
pcs = pd.read_csv("type_snp_size_100_weights_none_jobid_753502/all.pca.csv")

## MDS

In [ ]:
mds_coords = pd.read_csv("type_snp_size_100_weights_none_jobid_753502/mds_coords.csv")
mds_coords = pd.DataFrame(mds_coords.drop(columns=["chrom", "window"]))
labels = hdbscan.HDBSCAN(min_samples=30, min_cluster_size=2, cluster_selection_epsilon=0.03).fit_predict(mds_coords)
sum(labels == -1), sum(labels >= 0), max(labels)

In [ ]:
#for ms in range(5, 100, 5):
#    labels = hdbscan.HDBSCAN(min_samples=ms, min_cluster_size=2, cluster_selection_epsilon=0.03).fit_predict(mds_coords)
#    print(ms, sum(labels == -1), sum(labels >= 0), max(labels))

In [ ]:
#for cse in np.linspace(.0001, 0.1, 30):
#    for ms in range(5, 100, 5):
#        labels = hdbscan.HDBSCAN(min_samples=ms, min_cluster_size=2, cluster_selection_epsilon=float(cse)).fit_predict(mds_coords)
#        print(cse, ms, sum(labels == -1), sum(labels >= 0), max(labels))

In [ ]:
labels = hdbscan.HDBSCAN(min_samples=70, min_cluster_size=2, cluster_selection_epsilon=0.05).fit_predict(mds_coords)
sum(labels == -1), sum(labels >= 0), max(labels)

In [ ]:
px.scatter(x=mds_coords['MDS1'], y=mds_coords['MDS2'], color=labels)

In [ ]:
px.scatter(x=mds_coords['MDS3'], y=mds_coords['MDS2'], color=labels)

In [ ]:
px.scatter(x=mds_coords['MDS3'], y=mds_coords['MDS4'], color=labels)

In [ ]:
px.scatter(x=mds_coords['MDS5'], y=mds_coords['MDS4'], color=labels)

In [ ]:
mds_coords_unclustered = mds_coords[labels == -1]

In [ ]:
for ms in range(5, 100, 5):
    labels2 = hdbscan.HDBSCAN(min_samples=ms, min_cluster_size=2).fit_predict(mds_coords_unclustered)
    print(ms, sum(labels2 == -1), sum(labels2 >= 0), max(labels2))

In [ ]:
labels2 = hdbscan.HDBSCAN(min_samples=35, min_cluster_size=2).fit_predict(mds_coords_unclustered)
sum(labels2 == -1), sum(labels2 >= 0), max(labels2)

In [ ]:
px.scatter(x=mds_coords_unclustered['MDS1'], y=mds_coords_unclustered['MDS2'], color=labels2)

In [ ]:
labels2 = labels2 + labels[-1]
# labels = np.concatenate((labels, labels2))

In [ ]:
mds1 = np.concatenate((mds_coords['MDS1'], mds_coords_unclustered['MDS1']))
mds2 = np.concatenate((mds_coords['MDS2'], mds_coords_unclustered['MDS2']))
mds3 = np.concatenate((mds_coords['MDS3'], mds_coords_unclustered['MDS3']))
mds4 = np.concatenate((mds_coords['MDS4'], mds_coords_unclustered['MDS4']))
mds5 = np.concatenate((mds_coords['MDS5'], mds_coords_unclustered['MDS5']))
labels_combined = np.concatenate((labels, labels2))
px.scatter(x=mds1, y=mds2, color=labels_combined)

In [ ]:
px.scatter(x=mds3, y=mds2, color=labels_combined)

In [ ]:
px.scatter(x=mds3, y=mds4, color=labels_combined)

In [ ]:
px.scatter(x=mds5, y=mds4, color=labels_combined)

In [ ]:
mds_coords = pd.read_csv("type_snp_size_100_weights_none_jobid_753502/mds_coords.csv")
mds_coords = mds_coords.set_index(["chrom", "window"])
mds_coords

In [ ]:
data = mds_coords.stack().reset_index()
data

In [ ]:
data.columns = ["chrom", "window", "MDS", "val"]
#data = pd.DataFrame(data.drop(columns=["x"]))

In [ ]:
px.box(data, y="val", x="MDS")

In [ ]:
[stats.mean(data['val']), stats.stdev(data['val'])]

In [ ]:
upper = stats.mean(data['val']) + (4 * stats.stdev(data['val']))
lower = stats.mean(data['val']) - (4 * stats.stdev(data['val']))

In [ ]:
lower_matches = data[data['val'] <= lower]
upper_matches = data[data['val'] >= upper]

In [ ]:
lower_matches[lower_matches["chrom"] == "X22"]

In [ ]:
upper_matches[upper_matches["chrom"] == "X22"]

In [ ]:
chr8 = data[data['chrom'] == "X8"]
px.scatter(chr8, x="window", y="val", color="MDS", width=1024, height=600)

In [ ]:
windows = pd.read_csv("type_snp_size_100_weights_none_jobid_753502/X8.regions.csv")
windows

In [ ]:
windows.iloc[221]

In [ ]:
# Chr9 Peak -- 63098195
#windows[windows['end'] >= 63098195]

windows.values.tolist()[281]


In [ ]:
x8 = data[data['chrom'] == "X8"]
window = x9[x9['window'] == 584]

In [ ]:
window

In [ ]:
data[data['val'] >= upper]

## Examine specific PCAs

In [ ]:
pcs = pd.read_csv("type_snp_size_100_weights_none_jobid_753502/X8.pca.csv")
pcs = pcs.drop(columns=["total", "lam_1", "lam_2", "lam_3", "lam_4", "lam_5", "lam_6", "lam_7", "lam_8", "lam_9", "lam_10"])

pcs = pd.DataFrame(pcs.loc[221])
pcs = pcs.transpose()
pcs

In [ ]:
plot_pcs = pcs.filter(regex = 'PC_[\d+]').stack().reset_index().drop(columns=["level_0"])
plot_pcs.columns = ["PC_BIRD", "val"]


In [ ]:
split_pcs = plot_pcs['PC_BIRD'].str.split("_", n=2, expand = True)
plot_pcs['PC'] = split_pcs[1]
plot_pcs['Name'] = split_pcs[2]
plot_pcs.drop(columns=["PC_BIRD"], inplace = True)

In [ ]:
plot_pcs = plot_pcs.pivot(index="Name", columns="PC")
plot_pcs.columns = plot_pcs.columns.droplevel()

In [ ]:
plot_pcs['Name'] = plot_pcs.index

In [ ]:
fig = px.scatter(plot_pcs, x="1", y="2", text="Name", color=zephyr_offspring)
fig.update_traces(textposition='top center')
fig.show()


In [ ]:
plot_pcs[plot_pcs["Name"] == "Blake"]

In [ ]:
plot_pcs[plot_pcs["Name"] == "Sinbad"]

In [ ]:
ped = pd.read_csv("highconfidence.ped", sep="\t", header=None)
ped.columns = ["Pop", "Name", "Father", "Mother", "x", "y"]
zephyr_offspring = ped['Mother'] == "Zephyr"
sum(zephyr_offspring)

#rh_offspring = ped['Father'] == "Richard_Henry"
#sum(rh_offspring)

In [ ]:
plot_pcs[plot_pcs["Name"] == "Boomer"]

In [ ]:
plot_pcs[plot_pcs["Name"] == "Evohe"]

In [ ]:
plot_pcs[plot_pcs["Name"] == "Hoki"]

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(pcs)
embedding.shape

In [ ]:
px.scatter(x=embedding[:, 0], y=embedding[:, 1])

In [ ]:
plot_pcs

In [ ]:
plot_pcs = plot_pcs.reindex(columns=["Name", "1","2","3","4","5","6","7","8","9","10"])

In [ ]:
plot_pcs.to_csv("pcs_pheno.tsv", sep="\t", index=False)

## Haplotypes

In [ ]:
from collections import defaultdict
seqs = defaultdict(str)

haps = pd.read_csv("rir.tsv", sep="\t")
for hap in haps.values.tolist():
    for x in hap[4:]:
        split = x.split("=")
        name = split[0]
        
        alleles = re.split("[/\|]", split[1])
        i = 0
        
      # Probably not phased correctly here...
        for allele in alleles:
            if allele == ".":
                allele = "N"
            seqs[name + "_" + str(i)] = seqs[name + "_" + str(i)] + allele
            i = i +1


In [ ]:
for (id, seq) in seqs.items():
    print(">" + id)
    print(seq)

## PCA Clustering

In [ ]:
pcs = pd.read_csv("type_snp_size_100_weights_none_jobid_753502/all.pca.csv")

In [ ]:
pcs

In [ ]:
pcs = pcs.drop(columns=["total", "lam_1", "lam_2", "lam_3", "lam_4", "lam_5", "lam_6", "lam_7", "lam_8", "lam_9", "lam_10"])

In [ ]:
reducer = umap.UMAP(min_dist=0.0, random_state=1010)
embedding = reducer.fit_transform(pcs)
embedding.shape

In [ ]:
px.scatter(x=embedding[:, 0], y=embedding[:, 1])

In [ ]:
reducer = umap.UMAP(n_neighbors=3, min_dist=0.0, random_state=1010, n_components=2, metric='euclidean')
embedding = reducer.fit_transform(pcs)
embedding.shape

In [ ]:
px.scatter(x=embedding[:, 0], y=embedding[:, 1])

In [ ]:
labels = hdbscan.HDBSCAN(min_samples=50, min_cluster_size=2).fit_predict(pcs)

In [ ]:
sum(labels == -1), sum(labels >= 0), max(labels)

In [ ]:
reducer = umap.UMAP(n_neighbors=5, min_dist=0.0, random_state=1010, n_components=4, metric='euclidean')
embedding = reducer.fit_transform(pcs)
embedding.shape

In [ ]:
labels = hdbscan.HDBSCAN(min_samples=50, min_cluster_size=2).fit_predict(embedding)
sum(labels == -1), sum(labels >= 0), max(labels)

In [ ]:
clustering = SpectralClustering(n_clusters=2).fit_predict(mds_coords)

In [ ]:
px.scatter(x=embedding[:, 0], y=embedding[:, 1], color=clustering)

In [ ]:
output = list()
for n in range(2,40):
    reducer = umap.UMAP(n_neighbors=n, min_dist=0.0, random_state=1010, n_components=10, metric='euclidean')
    embedding = reducer.fit_transform(pcs)
    labels = hdbscan.HDBSCAN(min_samples=5, min_cluster_size=2).fit_predict(embedding)
    output.append([sum(labels == -1), sum(labels >= 0), max(labels), n])

In [ ]:
o = pd.DataFrame(output)

In [ ]:
px.scatter(x=o[0], y=o[2], color=o[3])

In [ ]:
embedding = MDS(n_components=4).fit_transform(pcs)

In [ ]:
px.scatter(x=embedding[:, 0], y=embedding[:, 1])

In [ ]:
reducer = umap.UMAP(n_neighbors=5, min_dist=0.0, random_state=1010, n_components=2, metric='euclidean')
umap_embedding = reducer.fit_transform(embedding)
umap_embedding.shape
labels = hdbscan.HDBSCAN(min_samples=5, min_cluster_size=2).fit_predict(embedding)

In [ ]:
px.scatter(x=umap_embedding[:, 0], y=umap_embedding[:, 1], color=labels)

In [ ]:
sum(labels == -1), sum(labels >= 0), max(labels)

## PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pcapca = pd.DataFrame(pca.fit_transform(pcs))

In [ ]:
px.scatter(x=pcapca[0], y=pcapca[1])

In [ ]:
labels = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=2).fit_predict(pcapca)

In [ ]:
sum(labels == -1), sum(labels >= 0), max(labels)

In [ ]:
mds_coords = pd.read_csv("type_snp_size_500_weights_none_jobid_033295/mds_coords.csv")

In [ ]:
mds_coords = pd.DataFrame(mds_coords.drop(columns=["chrom", "window"]))

In [ ]:
labels = hdbscan.HDBSCAN(min_samples=4, min_cluster_size=2).fit_predict(mds_coords)
sum(labels == -1), sum(labels >= 0), max(labels)

In [ ]:
px.scatter(x=mds_coords['MDS1'], y=mds_coords['MDS2'], color=labels)

In [ ]:
mds_coords

In [ ]:
reducer = umap.UMAP(n_neighbors=5, min_dist=0.0, random_state=1010, n_components=2, metric='euclidean')
umap_embedding = pd.DataFrame(reducer.fit_transform(mds_coords))
umap_embedding.shape
labels = hdbscan.HDBSCAN(min_samples=5, min_cluster_size=2).fit_predict(umap_embedding)

In [ ]:
sum(labels == -1), sum(labels >= 0), max(labels)

In [ ]:
px.scatter(x=umap_embedding[0], y=umap_embedding[1], color=labels)